# How to match? 

## 0. Import libraries 

In [28]:
# !pip install faiss-cpu
# !pip install lightautoml

In [30]:
!python --version

Python 3.8.16


In [31]:
import pandas as pd
import warnings
import numpy as np
from lightautoml.addons.digitaltwins import Matcher
warnings.filterwarnings('ignore')
%config Completer.use_jedi = False

## 1. Create or upload your dataset  
In this case we will create random dataset with known effect size  
If you have your own dataset, go to the part 2 


In [32]:
# Simulating dataset with known effect size
num_users = 10000
num_months = 12

signup_months = np.random.choice(np.arange(1, num_months), num_users) * np.random.randint(0,2, size=num_users) # signup_months == 0 means customer did not sign up
df = pd.DataFrame({
    'user_id': np.repeat(np.arange(num_users), num_months),
    'signup_month': np.repeat(signup_months, num_months), # signup month == 0 means customer did not sign up
    'month': np.tile(np.arange(1, num_months+1), num_users), # months are from 1 to 12
    'spend': np.random.poisson(500, num_users*num_months) #np.random.beta(a=2, b=5, size=num_users * num_months)*1000 # centered at 500
})
# A customer is in the treatment group if and only if they signed up
df["treat"] = df["signup_month"]>0
# Simulating an effect of month (monotonically decreasing--customers buy less later in the year)
df["spend"] = df["spend"] - df["month"]*10
# Simulating a simple treatment effect of 100
after_signup = (df["signup_month"] < df["month"]) & (df["treat"])
df.loc[after_signup,"spend"] = df[after_signup]["spend"] + 100
df.head()

,user_id,signup_month,month,spend,treat
0,0,0,1,491,False
1,0,0,2,472,False
2,0,0,3,457,False
3,0,0,4,449,False
4,0,0,5,469,False


In [33]:
# Setting the signup month (for ease of analysis)
i = 3
df_i_signupmonth = (
    df[df.signup_month.isin([0, i])]
    .groupby(["user_id", "signup_month", "treat"])
    .apply(
        lambda x: pd.Series(
            {
                "pre_spends": x.loc[x.month < i, "spend"].mean(),
                "post_spends": x.loc[x.month > i, "spend"].mean(),
            }
        )
    )
    .reset_index()
)
df_i_signupmonth

,user_id,signup_month,treat,pre_spends,post_spends
0,0,0,False,481.5,425.555556
1,2,0,False,524.0,421.333333
2,5,0,False,456.5,423.000000
3,8,0,False,475.5,420.333333
4,9,0,False,492.0,419.111111
...,...,...,...,...,...
5428,9984,0,False,496.0,418.888889
5429,9991,0,False,486.0,417.444444
5430,9992,0,False,458.5,426.777778
5431,9994,0,False,491.0,425.888889


In [34]:
# Additional category features
gender = np.random.choice(a=[0,1], size=df_i_signupmonth.user_id.nunique())
age = np.random.choice(a=range(18, 70), size=df_i_signupmonth.user_id.nunique())
industry = np.random.choice(a=range(1, 3), size=df_i_signupmonth.user_id.nunique())
df_i_signupmonth['age'] = age
df_i_signupmonth['is_male'] =  gender
df_i_signupmonth['industry'] =  industry
df_i_signupmonth['industry'] = df_i_signupmonth['industry'].astype('str')
df_i_signupmonth['treat'] = df_i_signupmonth['treat'].astype(int)
df_i_signupmonth.head()

,user_id,signup_month,treat,pre_spends,post_spends,age,is_male,industry
0,0,0,0,481.5,425.555556,26,0,2
1,2,0,0,524.0,421.333333,59,0,2
2,5,0,0,456.5,423.000000,40,1,1
3,8,0,0,475.5,420.333333,44,0,1
4,9,0,0,492.0,419.111111,58,1,2


In [35]:
df_i_signupmonth.columns

Index(['user_id', 'signup_month', 'treat', 'pre_spends', 'post_spends', 'age',
       'is_male', 'industry'],
      dtype='object')

## 2. Matching  
### 2.0 Init params
info_col used to define informative attributes that should not be part of matching, such as user_id  
But to explicitly store this column in the table, so that you can compare directly after computation

In [36]:
info_col = ['user_id', 'signup_month']

outcome = 'post_spends'
treatment = 'treat'

### 2.1 Simple matching
This is the easiest way to initialize and calculate metrics on a Matching task  
Use it when you are clear about each attribute or if you don't have any additional task conditions (Strict equality for certain features) 

In [37]:
# Standard model with base parameters
model = Matcher(input_data=df_i_signupmonth, outcome=outcome, treatment=treatment, info_col=info_col, n_neighbors=10)
results, quality_results = model.estimate()

[06.07.2023 10:40:00 | hypex | INFO]: Applying matching


Finding index
Done
Finding index
Done


[06.07.2023 10:40:00 | Faiss hypex | INFO]: Calculated the number of times each subject has appeared as a match: 426
[06.07.2023 10:40:00 | Faiss hypex | INFO]: Calculated the number of times each subject has appeared as a match: 5007
[06.07.2023 10:40:00 | Faiss hypex | INFO]: Estimating quality of matching
[06.07.2023 10:40:00 | metrics | INFO]: Standardised mean difference: age           0.001874
is_male       0.000000
pre_spends    0.001450
dtype: float64
[06.07.2023 10:40:00 | metrics | INFO]: Standardised mean difference: age           0.002332
is_male       0.000000
pre_spends    0.001670
dtype: float64
[06.07.2023 10:40:00 | metrics | INFO]: Kolmogorov-Smirnov test to check matching quality: 
{'age': 0.9999999957935777, 'is_male': 1.0, 'pre_spends': 0.9999999999981869}
[06.07.2023 10:40:00 | metrics | INFO]: Kolmogorov-Smirnov test to check matching quality: 
{'age': 0.12988908140786867, 'is_male': 1.0, 'pre_spends': 0.07204371439716685}
[06.07.2023 10:40:00 | psi_pandas | INFO

In [38]:
results

,effect_size,std_err,p-val,ci_lower,ci_upper
ATE,100.122577,0.565919,0.0,99.013376,101.231779
ATC,100.082721,0.575463,0.0,98.954814,101.210628
ATT,100.591028,0.694627,0.0,99.229559,101.952496


### 2.2 Matching with a fixed variable  
Used when you have categorical feature(s) that you want to compare by strict equality  
group_col is used for strict comparison of categorical features.  
In our case there is only one attribute  
If there are several such attributes, you should make one of them and use it

In [39]:
# group_col is used for strict comparison of categorical features.  
# In our case there is only one attribute  
# If there are several such attributes, you should make one of them and use it

group_col = "industry"

In [40]:
model = Matcher(input_data=df_i_signupmonth, outcome=outcome, treatment=treatment, 
                info_col=info_col, group_col=group_col)
results, quality_results = model.estimate()

[06.07.2023 10:40:07 | hypex | INFO]: Applying matching


Finding index
Done
Finding index
Done
Finding index
Done
Finding index
Done


[06.07.2023 10:40:07 | Faiss hypex | INFO]: Calculated the number of times each subject has appeared as a match: 426
[06.07.2023 10:40:07 | Faiss hypex | INFO]: Calculated the number of times each subject has appeared as a match: 5007
[06.07.2023 10:40:07 | Faiss hypex | INFO]: Estimating quality of matching
[06.07.2023 10:40:07 | metrics | INFO]: Standardised mean difference: age           0.002655
is_male       0.000000
pre_spends    0.002071
dtype: float64
[06.07.2023 10:40:07 | metrics | INFO]: Standardised mean difference: age           0.002825
is_male       0.000000
pre_spends    0.001863
dtype: float64
[06.07.2023 10:40:07 | metrics | INFO]: Kolmogorov-Smirnov test to check matching quality: 
{'age': 0.9999999957935777, 'is_male': 1.0, 'pre_spends': 0.9999999999981869}
[06.07.2023 10:40:07 | metrics | INFO]: Kolmogorov-Smirnov test to check matching quality: 
{'age': 0.13607122633435426, 'is_male': 1.0, 'pre_spends': 0.07583800923741527}
[06.07.2023 10:40:07 | psi_pandas | INFO

### 2.3 Matching but you don't know which features are better to use  
This is the easiest way to initialize and calculate metrics on a Matching task  
Use this method when you want to select the most important features and make matching based on them. 

In [41]:
model = Matcher(df=df_i_signupmonth, outcome=outcome, treatment=treatment, 
                info_col=info_col, group_col=group_col)

TypeError: __init__() got an unexpected keyword argument 'df'

In [14]:
features_importance = model.lama_feature_select()
features_importance

[20.06.2023 17:10:35 | digitaltwins | INFO]: Counting feature importance
[20.06.2023 17:10:35 | lama_feature_selector | INFO]: Getting feature scores


,Feature,Importance
0,pre_spends,80056.839844
3,industry,80056.839844
1,age,59525.629639
2,is_male,7849.370117


In [15]:
features = features_importance['Feature'].to_list()

In [16]:
# You can use both variant that you like. So you can just list the features to be matched

#results, quality_results = model.estimate(features=features_importance[:3])
results, quality_results = model.estimate(features=features[:3])

[20.06.2023 17:10:40 | digitaltwins | INFO]: Applying matching


Index(['pre_spends', 'age', 'treat', 'post_spends'], dtype='object')
Finding index
Done
Finding index
Done
Index(['pre_spends', 'age', 'treat', 'post_spends'], dtype='object')
Finding index
Done
Finding index
Done


[20.06.2023 17:10:41 | Faiss digitaltwins | INFO]: Calculated the number of times each subject has appeared as a match: 431
[20.06.2023 17:10:41 | Faiss digitaltwins | INFO]: Calculated the number of times each subject has appeared as a match: 5027
[20.06.2023 17:10:41 | Faiss digitaltwins | INFO]: Estimating quality of matching
[20.06.2023 17:10:41 | metrics | INFO]: Standardised mean difference: age           0.130605
is_male       0.105538
pre_spends    0.009410
dtype: float64
[20.06.2023 17:10:41 | metrics | INFO]: Standardised mean difference: age           0.077848
is_male       0.138916
pre_spends    0.060355
dtype: float64
[20.06.2023 17:10:41 | metrics | INFO]: Kolmogorov-Smirnov test to check matching quality: 
{'age': 0.04879199135327575, 'is_male': 0.007572879602437536, 'pre_spends': 0.7428867321956416}
[20.06.2023 17:10:41 | metrics | INFO]: Kolmogorov-Smirnov test to check matching quality: 
{'age': 7.716220057578571e-05, 'is_male': 1.8684872307385456e-12, 'pre_spends': 1

In [17]:
model.matcher.df_matched

,index,pre_spends,age,is_male,industry,user_id,signup_month,pre_spends_matched,age_matched,is_male_matched,user_id_matched,signup_month_matched,industry_matched,index_matched,post_spends,post_spends_matched,post_spends_matched_bias,treat,treat_matched
0,1933,488.0,46,0,1,3578,3,461.0,69.0,0.0,5498.0,0.0,2,[4747],535.555556,430.111111,105.757264,1,0
1,3911,484.5,34,0,1,7221,3,478.5,63.0,0.0,7549.0,0.0,2,[3404],510.000000,425.777778,84.222222,1,0
2,1945,469.0,68,1,1,3611,3,474.0,69.0,0.0,4681.0,0.0,1,[1309],523.333333,417.444444,105.584555,1,0
3,1825,486.0,49,1,1,3369,3,474.5,20.0,0.0,6851.0,0.0,2,[2880],532.000000,409.666667,122.017686,1,0
4,1839,498.0,54,1,1,3394,3,498.0,18.0,0.0,3263.0,0.0,1,[596],513.555556,418.222222,95.330505,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5022,2432,479.5,25,1,2,4516,0,503.0,57.0,0.0,6863.0,0.0,1,[1586],414.333333,520.333333,106.017303,0,1
5023,2433,504.5,65,0,2,4519,0,485.0,40.0,0.0,83.0,0.0,1,[4273],422.666667,523.111111,100.521979,0,1
5024,2434,484.0,48,1,2,4520,0,514.5,43.0,0.0,7542.0,3.0,1,[3427],419.666667,529.000000,109.330536,0,1
5025,2291,473.5,51,0,2,4275,0,479.0,50.0,0.0,4957.0,0.0,1,[5337],424.222222,522.777778,98.573964,0,1


## 3. Results  
### 3.1 ATE, ATT, ATC

In [18]:
# model.matcher.results
results

,effect_size,std_err,p-val,ci_lower,ci_upper
ATE,100.408545,0.586454,0.0,99.259094,101.557995
ATC,100.388934,0.596572,0.0,99.219652,101.558216
ATT,100.637278,0.708387,0.0,99.248840,102.025717


### 3.2 SMD, PSI, KS-test, repeats

In [19]:
# matching quality result - SMD
model.quality_result

{'psi':                 column  anomaly_score check_result                 column  \
 0          age_treated           0.07           OK          age_untreated   
 1     industry_treated           0.00           OK     industry_untreated   
 2  post_spends_treated          16.11          NOK  post_spends_untreated   
 3   pre_spends_treated           0.06           OK   pre_spends_untreated   
 
    anomaly_score check_result  
 0           0.03           OK  
 1           0.00           OK  
 2           8.28          NOK  
 3           0.02           OK  ,
 'ks_test':             match_control_to_treat  match_treat_to_control
 age                       0.048792            7.716220e-05
 is_male                   0.007573            1.868487e-12
 pre_spends                0.742887            1.432399e-05,
 'smd':             match_control_to_treat  match_treat_to_control
 age                       0.130605                0.077848
 is_male                   0.105538                0.138

In [20]:
# matching quality result - PSI
model.quality_result['psi']

,column,anomaly_score,check_result,column,anomaly_score,check_result
0,age_treated,0.07,OK,age_untreated,0.03,OK
1,industry_treated,0.00,OK,industry_untreated,0.00,OK
2,post_spends_treated,16.11,NOK,post_spends_untreated,8.28,NOK
3,pre_spends_treated,0.06,OK,pre_spends_untreated,0.02,OK


In [21]:
# matching quality result - KS-test

model.quality_result['ks_test']

,match_control_to_treat,match_treat_to_control
age,0.048792,7.716220e-05
is_male,0.007573,1.868487e-12
pre_spends,0.742887,1.432399e-05


In [22]:
# matching quality result - repeats
model.quality_result['repeats']

{'match_control_to_treat': 0.94, 'match_treat_to_control': 0.08}

### 3.3 Validation
validate result with placebo treatment  
our new effect size is close to zero (it should be)

In [23]:
 
model.validate_result()
 

[20.06.2023 17:10:41 | digitaltwins | INFO]: Applying validation of result


Index(['pre_spends', 'age', 'random_feature', 'treat', 'post_spends'], dtype='object')
Finding index
Done
Finding index
Done
Index(['pre_spends', 'age', 'random_feature', 'treat', 'post_spends'], dtype='object')
Finding index
Done
Finding index
Done


[20.06.2023 17:10:41 | Faiss digitaltwins | INFO]: Calculated the number of times each subject has appeared as a match: 431
[20.06.2023 17:10:41 | Faiss digitaltwins | INFO]: Calculated the number of times each subject has appeared as a match: 5027


Index(['pre_spends', 'age', 'random_feature', 'treat', 'post_spends'], dtype='object')
Finding index
Done
Finding index
Done
Index(['pre_spends', 'age', 'random_feature', 'treat', 'post_spends'], dtype='object')
Finding index
Done
Finding index
Done


[20.06.2023 17:10:41 | Faiss digitaltwins | INFO]: Calculated the number of times each subject has appeared as a match: 431
[20.06.2023 17:10:41 | Faiss digitaltwins | INFO]: Calculated the number of times each subject has appeared as a match: 5027


...

[20.06.2023 17:10:44 | Faiss digitaltwins | INFO]: Calculated the number of times each subject has appeared as a match: 431
[20.06.2023 17:10:44 | Faiss digitaltwins | INFO]: Calculated the number of times each subject has appeared as a match: 5027


Index(['pre_spends', 'age', 'random_feature', 'treat', 'post_spends'], dtype='object')
Finding index
Done
Finding index
Done
Index(['pre_spends', 'age', 'random_feature', 'treat', 'post_spends'], dtype='object')
Finding index
Done
Finding index
Done


[20.06.2023 17:10:44 | Faiss digitaltwins | INFO]: Calculated the number of times each subject has appeared as a match: 431
[20.06.2023 17:10:44 | Faiss digitaltwins | INFO]: Calculated the number of times each subject has appeared as a match: 5027


<class 'list'>


{'post_spends': [100.49348390369565, 0.29243691737365707]}

In [24]:
model.matcher.df_matched

,index,pre_spends,age,is_male,industry,user_id,signup_month,random_feature,pre_spends_matched,age_matched,...,user_id_matched,signup_month_matched,random_feature_matched,industry_matched,index_matched,post_spends,post_spends_matched,post_spends_matched_bias,treat,treat_matched
0,1933,488.0,46,0,1,3578,3,0.831283,486.5,27.0,...,6245.0,0.0,-0.853119,2,[4978],535.555556,424.333333,113.344309,1,0
1,3911,484.5,34,0,1,7221,3,0.280152,481.0,33.0,...,9520.0,0.0,-0.375390,2,[3220],510.000000,416.111111,93.931293,1,0
2,1945,469.0,68,1,1,3611,3,0.521268,474.5,69.0,...,5694.0,0.0,0.149171,1,[4577],523.333333,420.111111,101.065917,1,0
3,1825,486.0,49,1,1,3369,3,1.394904,493.5,22.0,...,7007.0,0.0,-0.379945,2,[2742],532.000000,416.333333,113.484695,1,0
4,1839,498.0,54,1,1,3394,3,-0.862996,482.0,44.0,...,7012.0,0.0,-0.339060,1,[2676],513.555556,417.222222,94.236638,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5022,2432,479.5,25,1,2,4516,0,-0.093153,489.0,32.0,...,487.0,0.0,1.245220,1,[1985],414.333333,514.555556,99.848473,0,1
5023,2433,504.5,65,0,2,4519,0,-1.189707,477.0,25.0,...,3905.0,0.0,0.316262,1,[5056],422.666667,513.222222,90.595060,0,1
5024,2434,484.0,48,1,2,4520,0,-2.577513,507.0,51.0,...,8530.0,0.0,0.487054,1,[701],419.666667,513.888889,94.139351,0,1
5025,2291,473.5,51,0,2,4275,0,-0.817607,461.5,44.0,...,6124.0,0.0,-1.190293,1,[4922],424.222222,523.111111,98.649767,0,1
